In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
plt.style.use("dark_background")

$$ \large f(x, y) = 3(1 - x)^2 e^{-x^2 - (y + 1)^2} - 10(\frac{x}{5} - x^3 - y^5) e^{-x^2 - y^2} - \frac{1}{3} e^{-(x + 1)^2 - y^2} $$

In [ ]:
def peaks(x,y):
  x,y = np.meshgrid(x,y)

  z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) \
      - 10*(x/5 - x**3 - y**5) * np.exp(-x**2-y**2) \
      - 1/3*np.exp(-(x+1)**2 - y**2)
  return z

x = np.linspace(-3,3, 201)
y = np.linspace(-3,3, 201)

Z = peaks(x,y)


In [ ]:
import plotly.graph_objects as go


# fig = go.Figure(data=[go.Surface(x=x, y=y, z=Z)])

# fig.update_layout(title='Peaks 2D Function', autosize=False,
#                   width=1000, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90),
#                   template="plotly_dark")


# fig.show()

In [ ]:
sx,sy = sp.symbols('sx,sy')

sZ = 3*(1-sx)**2 * sp.exp(-(sx**2) - (sy+1)**2) \
      - 10*(sx/5 - sx**3 - sy**5) * sp.exp(-sx**2-sy**2) \
      - 1/3*sp.exp(-(sx+1)**2 - sy**2)

df_x = sp.lambdify((sx,sy),sp.diff(sZ,sx),'sympy' ) #derivada de Z con respecto a X (sp.diff(funcion completa, simbolo a evaluar))
df_y = sp.lambdify((sx,sy),sp.diff(sZ,sy),'sympy' ) #derivada de Z con respecto a y

In [ ]:
from IPython.display import HTML
from matplotlib import animation
import PIL
import io

In [ ]:
def grad_2D(local_min_val=None, learning_rate=0.01, training_epochs=150, save=False):


    if local_min_val == None:
        local_min = np.random.rand(2)*4-2 # also try specifying coordinates
    else:
        local_min = local_min_val

    start_point = local_min[:] # make a copy, not re-assign

    print(f"Initial Random Local Min (x={local_min[0]}, y={local_min[1]})") if local_min_val == None else print(f"Initial Hardcoded Local Min (x={local_min[0]}, y={local_min[1]})")

    # training
    history = np.zeros((training_epochs,4))


    for i in range(training_epochs):


      grad = np.array([df_x(local_min[0],local_min[1]).evalf(),
                        df_y(local_min[0],local_min[1]).evalf()])

      history[i, 0] = local_min[0]
      history[i, 1] = local_min[1]

      history[i, 2] = grad[0]
      history[i, 3] = grad[1]

      local_min = local_min - learning_rate*grad

    print(f"Final Local Min after {training_epochs} epochs: (x={local_min[0]}, y={local_min[1]})")



 
    fig=go.Figure()

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0.1, y=0.1, z=1)
    )
    fig.add_trace(go.Surface(x=x, y=y, z=Z, colorscale="Viridis"))
    fig.add_trace(go.Scatter3d(x=np.array([history[0, 0]]),
                                        y=np.array([history[0, 1]]),
                                        z=peaks(history[0, 0], history[0, 1]).flatten(),
                                mode='markers', marker=dict(color="orange")))

    fig.update_layout(template="plotly_dark",
                        width=1200,  # Ancho de la figura
                        height=600,  # Altura de la figura
                        margin=dict(l=65, r=50, b=65, t=90),
                        );
    fig['data'][0].update(opacity=0.80)
    fig.update_traces(marker=dict(size=12,
                                    line=dict(width=10,
                                            color='DarkSlateGrey')),
                        selector=dict(mode='markers'))

    #go.Surface(z=Z),
    frames=[go.Frame(data=[go.Surface(x=x, y=y, z=Z, colorscale="Viridis"),
                            go.Scatter3d(x=np.array([history[k, 0]]),
                                        y=np.array([history[k, 1]]),
                                        z=(peaks(history[k, 0], history[k, 1])).flatten(),
                                        marker=dict(color="orange"))],
                        traces=[0,1]) for k in range(training_epochs)]

    fig.update(frames=frames)

    def frame_args(duration):
        return {
                "frame": {"duration": duration},
                "mode": "immediate",
                "redraw" : True,
                "fromcurrent": True,
                "transition": {"duration": duration, "easing": "linear"},
            }

    fr_duration=100
    sliders = [
                {
                    "pad": {"b": 10, "t": 50},
                    "len": 0.9,
                    "x": 0.1,
                    "y": 0,
                    "steps": [
                        {
                            "args": [[f.name], frame_args(fr_duration)],
                            "label": f"epoch{k+1}",
                            "method": "animate",
                        }
                        for k, f in enumerate(fig.frames)
                    ],
                }
            ]


    fig.update_layout(sliders=sliders, scene_camera=camera,

                        updatemenus = [
                            {
                            "buttons": [
                                {
                                    "args": [None, frame_args(fr_duration)],
                                    "label": "&#9654;", # play symbol
                                    "method": "animate",
                                },
                                {
                                    "args": [[None], frame_args(fr_duration)],
                                    "label": "&#9724;", # pause symbol
                                    "method": "animate",
                                }],
                            "direction": "left",
                            "pad": {"r": 10, "t": 70},
                            "type": "buttons",
                            "x": 0.1,
                            "y": 0,
                            }])

    if save:
        # generate images for each step in animation
        frames = []
        for s, fr in enumerate(fig.frames):
            # set main traces to appropriate traces within plotly frame
            fig.update(data=fr.data)
            # move slider to correct place
            fig.layout.sliders[0].update(active=s)
            # generate image of current state
            frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))

        # append duplicated last image more times, to keep animation stop at last status
        for i in range(5):
            frames.append(frames[-1])

        # create animated GIF
        frames[0].save(
                "test.gif",
                save_all=True,
                append_images=frames[1:],
                optimize=False,
                duration=500,
                loop=0,
            )


    return fig
        # ani.save('gradient1D.gif', writer=writer)
        # from IPython.display import HTML
            # HTML(anim.to_html5_video())

In [ ]:
fig = grad_2D(save=False)

We can rotate and interact with this rendered 3D plot

In [ ]:
fig.show()